In [131]:
from github import Github
import pandas as pd
import os
from distutils.version import LooseVersion
import numpy
from datetime import datetime
import time
import psycopg2
import re

In [136]:
links = pd.read_csv('complete_links_allVersions.csv')
links = links.sort_values(['project_name', 'version_name'], ascending=False)
links.head()

,project_name,project_link,version_name,commit
2884,web3j-web3j,https://github.com/web3j/web3j,v5.0.0,"Commit(sha=""d1810a1cbe339cb9038ee477886601b849..."
2885,web3j-web3j,https://github.com/web3j/web3j,v4.6.1,"Commit(sha=""8e94d2bcf305358d030bab9f89d4692bb3..."
2887,web3j-web3j,https://github.com/web3j/web3j,v4.6.0-android,"Commit(sha=""9ff030a628caf2c3a77e8952bbf7feb890..."
2886,web3j-web3j,https://github.com/web3j/web3j,v4.6.0,"Commit(sha=""5877c308e11c0de8e24e50c882effa8435..."
2897,web3j-web3j,https://github.com/web3j/web3j,v4.5.9,"Commit(sha=""6c23f373ce8c30a38331c4eebf46517be5..."


### Get major version based on split 5.0.0 = 5

In [165]:
#links['major_version'] = links['version_name'].apply(lambda x: x.split('-')[-1])
links['major_version'] = links['version_name'].apply(lambda x: x.split('.')[0])
links['major_version'] = links['major_version'].apply(lambda x: x.split('-')[-1])
links['major_version'] = links['major_version'].apply(lambda x: x.split('/')[-1])
links['major_version'] = links['major_version'].apply(lambda x: x.split('_')[-1])
links['major_version'] = links['major_version'].apply(lambda x: x.replace('v',''))
links['major_version'] = links['major_version'].apply(lambda x: x.replace('r',''))
links['major_version_int'] = pd.to_numeric(links['major_version'],errors='coerce')

In [138]:
links.head(5)

,project_name,project_link,version_name,commit,major_version
2884,web3j-web3j,https://github.com/web3j/web3j,v5.0.0,"Commit(sha=""d1810a1cbe339cb9038ee477886601b849...",v5
2885,web3j-web3j,https://github.com/web3j/web3j,v4.6.1,"Commit(sha=""8e94d2bcf305358d030bab9f89d4692bb3...",v4
2887,web3j-web3j,https://github.com/web3j/web3j,v4.6.0-android,"Commit(sha=""9ff030a628caf2c3a77e8952bbf7feb890...",v4
2886,web3j-web3j,https://github.com/web3j/web3j,v4.6.0,"Commit(sha=""5877c308e11c0de8e24e50c882effa8435...",v4
2897,web3j-web3j,https://github.com/web3j/web3j,v4.5.9,"Commit(sha=""6c23f373ce8c30a38331c4eebf46517be5...",v4


In [139]:
view = links[links['project_name'] == 'socketio-socket.io-client-java']
view.head()

,project_name,project_link,version_name,commit,major_version
1839,socketio-socket.io-client-java,https://github.com/socketio/socket.io-client-java,socket.io-client-1.0.0,"Commit(sha=""45fd4fbdd8401ea63c30f6bbda266afd99...",socket
1840,socketio-socket.io-client-java,https://github.com/socketio/socket.io-client-java,socket.io-client-0.9.0,"Commit(sha=""dbd9553732477424114ea1ac20709bb852...",socket
1841,socketio-socket.io-client-java,https://github.com/socketio/socket.io-client-java,socket.io-client-0.8.3,"Commit(sha=""fcee9964188de49752923816114a3f9ed9...",socket
1842,socketio-socket.io-client-java,https://github.com/socketio/socket.io-client-java,socket.io-client-0.8.2,"Commit(sha=""582fbe4a7d2f68c83f1737de4e1cdcdbac...",socket
1843,socketio-socket.io-client-java,https://github.com/socketio/socket.io-client-java,socket.io-client-0.8.1,"Commit(sha=""b94abcaf6634ddf70cc83ae64fbf0b6c7f...",socket


### Aggregate count of major versions

In [140]:
cleaned_links = links.groupby(['project_name','major_version']).agg(['count']).reset_index()
cleaned_links.columns = cleaned_links.columns.droplevel()
cleaned_links.columns = ['project_name', 'major_version', 'count', 'count1', 'count2']
cleaned_links = cleaned_links[['project_name', 'major_version', 'count']]
cleaned_links.head()

,project_name,major_version,count
0,Flow86-Additional-Buildcraft-Objects-Mod,3,1
1,Flow86-Additional-Buildcraft-Objects-Mod,jenkins-1,5
2,Flow86-Additional-Buildcraft-Objects-Mod,jenkins-11,1
3,Flow86-Additional-Buildcraft-Objects-Mod,jenkins-13,1
4,Flow86-Additional-Buildcraft-Objects-Mod,jenkins-15,1


### Get count that are more than 20

In [155]:
cleaned_links_requirement_fulfill = cleaned_links[cleaned_links['count'] >= 20]
cleaned_links_requirement_fulfill = cleaned_links_requirement_fulfill.sort_values(['project_name', 'major_version'], ascending=False)
cleaned_links_requirement_fulfill = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['project_name'] != 'socketio-socket.io-client-java']
cleaned_links_requirement_fulfill = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['project_name'] != 'apache-sling-org-apache-sling-testing-sling-mock']
cleaned_links_requirement_fulfill = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['project_name'] != 'apache-sling-org-apache-sling-scripting-jsp']
cleaned_links_requirement_fulfill = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['project_name'] != 'apache-sling-org-apache-sling-resourceresolver']
cleaned_links_requirement_fulfill = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['project_name'] != 'apache-sling-org-apache-sling-event']
cleaned_links_requirement_fulfill['major_version'] = cleaned_links_requirement_fulfill['major_version'].apply(lambda x: x.split('-')[-1])
cleaned_links_requirement_fulfill['major_version'] = cleaned_links_requirement_fulfill['major_version'].apply(lambda x: x.split('/')[-1])
cleaned_links_requirement_fulfill['major_version'] = cleaned_links_requirement_fulfill['major_version'].apply(lambda x: x.split('_')[-1])
cleaned_links_requirement_fulfill['major_version'] = cleaned_links_requirement_fulfill['major_version'].apply(lambda x: x.replace('v',''))
cleaned_links_requirement_fulfill['major_version'] = cleaned_links_requirement_fulfill['major_version'].apply(lambda x: x.replace('r',''))
cleaned_links_requirement_fulfill['major_version'] = pd.to_numeric(cleaned_links_requirement_fulfill['major_version'])
cleaned_links_requirement_fulfill['count'] = cleaned_links_requirement_fulfill['count'].astype(str)
cleaned_links_requirement_fulfill['rank'] = cleaned_links_requirement_fulfill.groupby('project_name').rank(method='min',ascending=False)
cleaned_links_requirement_fulfill.head(5)

,project_name,major_version,count,rank
3545,web3j-web3j,4,30,1.0
3537,voxeolabs-moho,0,48,2.0
3536,voxeolabs-moho,1,27,1.0
3534,uber-NullAway,0,52,1.0
3528,testcontainers-testcontainers-java,1,48,1.0


In [161]:
project_links_rank_1 = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['rank'] == 1]
project_links_rank_1['major_version_int'] = project_links_rank_1['major_version']
project_links_rank_1.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,project_name,major_version,count,rank,major_version_int
3545,web3j-web3j,4,30,1.0,4
3536,voxeolabs-moho,1,27,1.0,1
3534,uber-NullAway,0,52,1.0,0
3528,testcontainers-testcontainers-java,1,48,1.0,1
3525,swagger-api-swagger-core,2,20,1.0,2


In [160]:
project_links_rank_1.dtypes

project_name      object
major_version      int64
count             object
rank             float64
dtype: object

In [170]:
final_links = pd.merge(links, project_links_rank_1, how='right', on=['project_name', 'major_version_int'])
final_links.tail()

,project_name,project_link,version_name,commit,major_version_x,major_version_int,major_version_y,count,rank
46586,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.10.1,"Commit(sha=""6a1fa20ca409f6e74fe1b46898671730db...",2,2.0,2,29,1.0
46587,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.10,"Commit(sha=""0ae53118955118d4618ccf8ae7cf1b6d96...",2,2.0,2,29,1.0
46588,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.1,"Commit(sha=""67178f0d41ac94afd03042c6c4ab17cec0...",2,2.0,2,29,1.0
46589,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.0-RC1,"Commit(sha=""47da336524c37da3673183f02b56a9c7fb...",2,2.0,2,29,1.0
46590,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.0,"Commit(sha=""c5a5190e19c062405ae4825c36e2172ae6...",2,2.0,2,29,1.0


In [171]:
final_links.to_csv('rawFinalProjectLinksLatestFromSameMajorRelease.csv', index=False)

In [172]:
final_links

,project_name,project_link,version_name,commit,major_version_x,major_version_int,major_version_y,count,rank
0,web3j-web3j,https://github.com/web3j/web3j,v4.6.1,"Commit(sha=""8e94d2bcf305358d030bab9f89d4692bb3...",4,4.0,4,30,1.0
1,web3j-web3j,https://github.com/web3j/web3j,v4.6.0-android,"Commit(sha=""9ff030a628caf2c3a77e8952bbf7feb890...",4,4.0,4,30,1.0
2,web3j-web3j,https://github.com/web3j/web3j,v4.6.0,"Commit(sha=""5877c308e11c0de8e24e50c882effa8435...",4,4.0,4,30,1.0
3,web3j-web3j,https://github.com/web3j/web3j,v4.5.9,"Commit(sha=""6c23f373ce8c30a38331c4eebf46517be5...",4,4.0,4,30,1.0
4,web3j-web3j,https://github.com/web3j/web3j,v4.5.8,"Commit(sha=""62df9d1c91281664c8534b994a4bbf0941...",4,4.0,4,30,1.0
...,...,...,...,...,...,...,...,...,...
46586,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.10.1,"Commit(sha=""6a1fa20ca409f6e74fe1b46898671730db...",2,2.0,2,29,1.0
46587,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.10,"Commit(sha=""0ae53118955118d4618ccf8ae7cf1b6d96...",2,2.0,2,29,1.0
46588,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.1,"Commit(sha=""67178f0d41ac94afd03042c6c4ab17cec0...",2,2.0,2,29,1.0
46589,JodaOrg-joda-time,https://github.com/JodaOrg/joda-time,v2.0-RC1,"Commit(sha=""47da336524c37da3673183f02b56a9c7fb...",2,2.0,2,29,1.0


In [175]:
final_20limit_links = final_links.groupby('project_name').head(20)
final_20limit_links.to_csv('complete_links_allVersions_limit20.csv', index=False)


In [176]:
print(len(final_20limit_links))

2960


In [145]:
test2 = cleaned_links_requirement_fulfill[cleaned_links_requirement_fulfill['major_version'].apply(lambda x: not x.isnumeric())]
print(len(test2))
test2.head(7)

0


,project_name,major_version,count


In [98]:
print(len(cleaned_links_requirement_fulfill))
print(len(cleaned_links_requirement_fulfill['project_name'].unique()))


286
153


In [87]:
cleaned_links_requirement_fulfill.groupby('project_name')

In [32]:
test = links[links['project_name'] == 'web3j-web3j']

81


,project_name,project_link,version_name,commit
2884,web3j-web3j,https://github.com/web3j/web3j,v5.0.0,"Commit(sha=""d1810a1cbe339cb9038ee477886601b849..."
2885,web3j-web3j,https://github.com/web3j/web3j,v4.6.1,"Commit(sha=""8e94d2bcf305358d030bab9f89d4692bb3..."
2887,web3j-web3j,https://github.com/web3j/web3j,v4.6.0-android,"Commit(sha=""9ff030a628caf2c3a77e8952bbf7feb890..."
2886,web3j-web3j,https://github.com/web3j/web3j,v4.6.0,"Commit(sha=""5877c308e11c0de8e24e50c882effa8435..."
2897,web3j-web3j,https://github.com/web3j/web3j,v4.5.9,"Commit(sha=""6c23f373ce8c30a38331c4eebf46517be5..."
2898,web3j-web3j,https://github.com/web3j/web3j,v4.5.8,"Commit(sha=""62df9d1c91281664c8534b994a4bbf0941..."
2899,web3j-web3j,https://github.com/web3j/web3j,v4.5.7,"Commit(sha=""24e7d1ef271141f7081bc73d26e6268836..."
2900,web3j-web3j,https://github.com/web3j/web3j,v4.5.6,"Commit(sha=""93eed33dfbe32bb39113f2270115a3e1f6..."
2901,web3j-web3j,https://github.com/web3j/web3j,v4.5.5,"Commit(sha=""b58beb105f49b77bf08df09a9c01f07106..."
2902,web3j-web3j,https://github.com/web3j/web3j,v4.5.4,"Commit(sha=""4a595a7a1354dd1894b1e20c0066236f9c..."


In [31]:
print(len(links['version_name'].unique()))

23420
